# Data Extraction module

Imports

In [1]:
import os
from tqdm import tqdm

from data_access.insert.star import insert_star_type
from data_access.read.star import star_exists
from tools.normalize import create_normalized_light_curve_from_fits, get_sector_data_folder, get_tic_index_from_full_name
from data_access.insert.sector import insert_file_name_pattern_to_sector
from data_access.insert.flux import insert_flux
from data_access.read.flux import flux_exists
from tools.star_type import get_star_type

### Variables Selection

In [2]:
sector = 5

### Process fits files and save fluxes to DB

In [3]:
folder = get_sector_data_folder(sector)
all_files = os.listdir(folder)

# save file name pattern for sector to DB
first_file = all_files[0]
file_parts = first_file.split('-')
file_parts[2] = '{}'
file_name_pattern = '-'.join(file_parts)
insert_file_name_pattern_to_sector(sector, file_name_pattern)

star_tics = [str(get_tic_index_from_full_name(file)) for file in all_files]

### Insert data about stars

In [4]:
for star_tic in tqdm(star_tics):
    if star_exists(star_tic):
        continue

    star_type = get_star_type(star_tic)
    insert_star_type(star_tic, star_type)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]C:\Users\denys.karakuts\PycharmProjects\capstone_project\tools\star_type.py:28: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 28 of the file C:\Users\denys.karakuts\PycharmProjects\capstone_project\tools\star_type.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html_content)
C:\Users\denys.karakuts\AppData\Local\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML documen

### Insert data about star luminescence

In [5]:
for star_tic in tqdm(star_tics):
    if flux_exists(sector, star_tic):
        continue

    normalized_light_curve = create_normalized_light_curve_from_fits(sector, star_tic)
    insert_flux(sector, star_tic, normalized_light_curve)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 10.30it/s]
